In [1]:
import pandas as pd
from pymongo import MongoClient
client = MongoClient()
import json
from pandas.io.json import json_normalize
from geojson import Polygon

Se solicitan, de la base de datos de mongo, aquellos datos en los cuales exista latitud y longitud y las columnas que son de interés.

In [2]:
db = client.companies

x=db.companies.find({"$and":
                     [ {'offices.latitude':{'$exists': True, '$ne':None}},
                      {'offices.longitude':{'$exists': True,'$ne':None}},
                      {'founded_year':{'$gt':2000}},
                     ]} 
                    ,{'name':1, 'category_code':1, 'number_of_employees':1, 'offices.latitude':1, 'offices.longitude':1,'offices.city':1,'founded_year':1,'relationships.title':1,'_id':0})



A continuación se normalizan los datos de offices para aplanarlos. 

In [3]:
df = json_normalize (data=x, record_path=['offices'], 
                     meta=['name', 'category_code','number_of_employees','founded_year'], 
                     errors='ignore')

In [4]:
df.head()

,city,latitude,longitude,name,category_code,number_of_employees,founded_year
0,Seattle,47.603122,-122.333253,Wetpaint,web,47,2005
1,New York,40.723731,-73.996431,Wetpaint,web,47,2005
2,Pleasanton,37.692934,-121.904945,Zoho,software,1600,2005
3,West Hollywood,34.090368,-118.393064,Geni,web,18,2006
4,San Francisco,37.764726,-122.394523,Digg,news,60,2004


A continuación se comprueba si hay ciudades con pocas empresas.

In [5]:
df['city'].value_counts()

San Francisco             441
New York                  349
London                    194
                          167
Seattle                   134
Austin                    110
Palo Alto                 106
Los Angeles                98
Mountain View              94
Chicago                    89
Sunnyvale                  77
Paris                      75
Toronto                    68
San Diego                  66
San Jose                   65
San Mateo                  61
Cambridge                  57
Atlanta                    57
Boston                     56
Berlin                     52
Santa Clara                51
Redwood City               43
Vancouver                  40
Menlo Park                 39
Santa Monica               37
Boulder                    36
Amsterdam                  36
Bangalore                  35
Portland                   31
Madrid                     30
                         ... 
Longueuil                   1
Baghdad                     1
Bear      

Puesto que en algunas ciudades hay pocas empresas, y queremos lugares en los cuales tengamos varias empresas cerca, se eliminan todas aquellas ciudades que tengan menos de 20 empresas.

In [6]:
df=df.groupby('city').filter(lambda x : len(x)>20)

A continuación se observa los tipos de compañía registrados.

In [7]:
df['category_code'].value_counts()

web                 836
software            418
games_video         259
advertising         232
mobile              226
enterprise          190
ecommerce           140
other               124
network_hosting     121
search               98
public_relations     95
consulting           77
cleantech            51
hardware             44
biotech              34
security             31
social               30
analytics            27
semiconductor        20
finance              18
education            14
music                14
news                 13
travel               12
messaging            11
medical               7
health                6
real_estate           6
legal                 4
hospitality           4
photo_video           4
design                2
manufacturing         2
automotive            2
nanotech              2
fashion               2
sports                2
nonprofit             2
Name: category_code, dtype: int64

En vista de que las categorías menos repetidas no son el tipo de categorías que nos resulta de interés, se eliminan también todas las repetidas menos de 20 veces.

In [8]:
df=df.groupby('category_code').filter(lambda x : len(x)>20)

Se resetea el índice para que vuelva a estar ordenado y se pueda trabajar con ello.

In [9]:
df=df.reset_index()
df.head()

,index,city,latitude,longitude,name,category_code,number_of_employees,founded_year
0,0,Seattle,47.603122,-122.333253,Wetpaint,web,47,2005
1,1,New York,40.723731,-73.996431,Wetpaint,web,47,2005
2,5,Menlo Park,37.484130,-122.169472,Gizmoz,web,None,2003
3,7,Los Angeles,34.057498,-118.446596,Helio,mobile,None,2005
4,9,New York City,40.757929,-73.985506,MeetMoi,social,15,2007


In [10]:
df = df.drop('index',axis=1)

Se crea el punto de geolocalización para utilizar a través de mongo. 

In [11]:
serie=[]
index=0
for i in df['latitude']:
    serie.append({"type": "Point", "coordinates": [df["longitude"][index], df["latitude"][index]]})
    index+=1
    
df['geo']=serie
df.head()

,city,latitude,longitude,name,category_code,number_of_employees,founded_year,geo
0,Seattle,47.603122,-122.333253,Wetpaint,web,47,2005,"{'type': 'Point', 'coordinates': [-122.333253,..."
1,New York,40.723731,-73.996431,Wetpaint,web,47,2005,"{'type': 'Point', 'coordinates': [-73.9964312,..."
2,Menlo Park,37.484130,-122.169472,Gizmoz,web,None,2003,"{'type': 'Point', 'coordinates': [-122.169472,..."
3,Los Angeles,34.057498,-118.446596,Helio,mobile,None,2005,"{'type': 'Point', 'coordinates': [-118.446596,..."
4,New York City,40.757929,-73.985506,MeetMoi,social,15,2007,"{'type': 'Point', 'coordinates': [-73.985506, ..."


Se guarda en csv para tableau y en json para mongo. 

In [12]:
df.to_csv('./companies.csv')

In [13]:
df.to_json('./companies.json', orient='records', lines=True)

En tableau se puede ver el mapa representando la localización de todas las empresas obtenidas tras esta primera limpieza.
URL:  https://public.tableau.com/profile/patricia.rick#!/vizhome/Companies_distribution/Dashboard1?publish=yes